# Listas y como recorrerlas:

Comenzamos estudiando los arrays y las formas de recorrerlos

In [ ]:
# array
arr1 = ["algebra 1", "calculo 1", "matematica discreta"]
print("El tipo de objeto es: ",type(arr1))


El tipo de objeto es:  <class 'list'>


Los arrays en python son objetos tipo "lista", que tienen sus funciones (append,insert, etc).

Para recorrerlo y trabajar con el:

In [2]:
for i in arr1: # i es un ELEMENTO del array, no un índice
  print(i)



algebra 1
calculo 1
matematica discreta


In [3]:
# tambien se puede usar una estructura mas tradicional:

for i in range(len(arr1)):
  print(f"índice: {i}, elemento: {arr1[i]}")



índice: 0, elemento: algebra 1
índice: 1, elemento: calculo 1
índice: 2, elemento: matematica discreta


In [4]:
# otra forma:

for i, materia in enumerate(arr1): # una forma de combinar ambas formas de recorrer
  print(f"valor de i: {i}, materia: {materia}")

valor de i: 0, materia: algebra 1
valor de i: 1, materia: calculo 1
valor de i: 2, materia: matematica discreta


Existen diversas formas de agregar elementos a arrays


In [5]:
# agregamos la mejor matematica de ela catedra:

arr1.append("Optimización con Álgebra Lineal")
print("Array original, pero con la mejor matemática agregada al final:",arr1)

# si queremos obtener los elementos del 1 al 2:

print("Elementos del 1 al 3: ",arr1[1:3]) # no incluye al 3. Numericamente es comodo

# si queremos el ultimo elemento:
print("Ultimo elemento:",arr1[-1])

# si queremos los elementos del 2 al ultimo:
print("elementos del 3 en adelante:",arr1[2:])

Array original, pero con la mejor matemática agregada al final: ['algebra 1', 'calculo 1', 'matematica discreta', 'Optimización con Álgebra Lineal']
Elementos del 1 al 3:  ['calculo 1', 'matematica discreta']
Ultimo elemento: Optimización con Álgebra Lineal
elementos del 3 en adelante: ['matematica discreta', 'Optimización con Álgebra Lineal']


Para trabajar con arrays numéricos, recomendamos utilizar los objetos "numpy array" que vienen con sus propias funciones mas orientadas al algebra

In [ ]:
import numpy as np
arr2 = [1, 2, 3, 4, 5]
arr2 = np.array(arr2) # lo convertimos a numpy array.

# arr2 mantiene todas las propiedades de una lista normal, pero tenemos mas funciones.


print("El tipo es:",type(arr2))


El tipo es: <class 'numpy.ndarray'>


In [ ]:
# arr2 es compatible con operaciones matematicas

print("Elementos de arr2 al cuadrado: ",np.power(arr2, 2))

# imprimir solo los numeros pares:

print("Elementos pares de arr2:")
for i in arr2:
  if i%2 == 0:
    print(i)





Elementos de arr2 al cuadrado:  [ 1  4  9 16 25]
Elementos pares de arr2:
2
4


In [ ]:
# otra forma de hacerlo:
# para python, 0 es igual a FALSE y cualquier cosa distinto de 0 es igual a TRUE, podemos hacer uso de eso
print("Elementos pares de arr2")
for i in arr2:
  if not i%2:
    print(i)

# si queremos los impares:
print("Elementos impares de arr2")
for i in arr2:
  if i%2: # si i%2 es 0, entonces la condicion es FALSE y no entra. Si i%2 es 1 (impar) entonces es TRUE y entra
    print(i)


Elementos pares de arr2
2
4
Elementos impares de arr2
1
3
5


# Importar archivos csv


Primero montaremos una cuenta de google drive. Hay que elegir la cuenta donde hayan subido los archivos csv.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


En Python, los datasets se suelen trabajar con la librería Pandas, la cual ofrece métodos para manipular las tablas de datos. A continuación se muestra el código inicial para cargar los datos del problema 1.

In [ ]:
# Importamos las librerías necesarias
import cvxpy as cp
import numpy as np
import pandas as pd

# Cargamos los datos desde los archivos CSV
costo_ancho_de_banda_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/datos/costo_ancho_de_banda.csv')
print("Costo de ancho de banda:")
print(costo_ancho_de_banda_df)
latencia_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/datos/latencia.csv')
print("Latencias:")
print(latencia_df)
demanda_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/datos/demanda_usuarios.csv')
print("Demanda:")
print(demanda_df)
carga_maxima_servidores_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/datos/carga_maxima_servidores.csv')
print("Carga máxima de los servidores:")
print(carga_maxima_servidores_df)

# Extraemos las listas únicas de grupos de usuarios y servidores
grupos_usuarios = demanda_df['grupo de usuarios'].unique()
servidores = costo_ancho_de_banda_df['servidor'].unique()

# Creamos diccionarios para un acceso más rápido a los datos
demanda = dict(zip(demanda_df['grupo de usuarios'], demanda_df['demanda (solicitudes por minuto)']))
print("Demanda (dict):")
print(demanda)

# Transformamos los DataFrames de costos y latencias en tablas pivot
costo_ancho_de_banda = costo_ancho_de_banda_df.pivot(
    index='grupo de usuarios',
    columns='servidor',
    values='costo del ancho de banda por solicitud (USD por solicitud)'
).fillna(0)

latencia = latencia_df.pivot(
    index='grupo de usuarios',
    columns='servidor',
    values='tiempo de respuesta promedio por solicitud (milisegundos)'
).fillna(0)

# Aseguramos que los valores de latencia y costo sean numéricos
latencia = latencia.apply(pd.to_numeric, errors='coerce').fillna(0)
costo_ancho_de_banda = costo_ancho_de_banda.apply(pd.to_numeric, errors='coerce').fillna(0)

# Leemos la capacidad máxima de los servidores
S_j = dict(zip(carga_maxima_servidores_df['servidor'], carga_maxima_servidores_df['carga maxima (solicitudes por minuto)']))

# Parámetros del SLA
L_SLA = 400  # Umbral de latencia en milisegundos
k = 0.0000001  # Costo por milisegundo excedido por solicitud (USD por solicitud por ms)
# La última linea que se pone en una notebook de colab se imprime.

Costo de ancho de banda:
    grupo de usuarios  servidor  \
0                   1         1   
1                   1         2   
2                   1         3   
3                   1         4   
4                   1         5   
5                   1         6   
6                   2         1   
7                   2         2   
8                   2         3   
9                   2         4   
10                  2         5   
11                  2         6   
12                  3         1   
13                  3         2   
14                  3         3   
15                  3         4   
16                  3         5   
17                  3         6   
18                  4         1   
19                  4         2   
20                  4         3   
21                  4         4   
22                  4         5   
23                  4         6   
24                  5         1   
25                  5         2   
26                  5         

In [ ]:
# Ejemplo de cómo definir una función
def seridor_cumple_SLA_con_grupo(i, j):
  v = latencia.iloc[i,j]
  if v <= 400:
    print(f"El servidor {servidores[i]} cumple con el SLA para el grupo de usuarios {grupos_usuarios[j]}") # Los "f-strings" de python pemiten concatenar valores poniendolos entre {}
  else:
    print(f"El servidor {servidores[i]} NO cumple con el SLA para el grupo de usuarios {grupos_usuarios[j]}")


# Una vez definida la función, puedo usarla
seridor_cumple_SLA_con_grupo(1,2)
seridor_cumple_SLA_con_grupo(2,4)
seridor_cumple_SLA_con_grupo(3,3)

El servidor 2 NO cumple con el SLA para el grupo de usuarios 3
El servidor 3 NO cumple con el SLA para el grupo de usuarios 5
El servidor 4 cumple con el SLA para el grupo de usuarios 4


In [ ]:
# @ es el operador de producto matricial en numpy:
c1 = np.matrix([[1,0]
              ,[0,1]])
c2 = np.matrix([[1,1]
              ,[0,1]])
print(c1 @ c2)
print("\n") # salto de linea
print(c1 @ c2.T) # .T hace la traspuesta

[[1 1]
 [0 1]]


[[1 0]
 [1 1]]


# Ejemplo: ejercicio 3 del parcial de 2022
$$A =
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{-1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\end{pmatrix}$$

Problema:
* Probar que $A^{8} = I$
* Hallar valores y vectores propios

In [ ]:
# Parte 1


a = np.matrix([[1, -1],[1,1]]) # defino el objeto matriz
const = np.divide(1,np.sqrt(2)) # defino el 1/sqrt(2) (puedo hacer 1/np.sqrt(2) tambien)
a = np.dot(a, const) # multiplico la matriz por 1/sqrt(2). Uso la funcion np.dot (puedo hacer a*const tambien)


print(f"La matriz es: \n {a}")
print("\n")
print(f"La matriz elevada a la 8 es: \n {np.linalg.matrix_power(a, 8)}") # elevo la matriz a la 8


La matriz es: 
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


La matriz elevada a la 8 es: 
 [[ 1.00000000e+00  2.39116835e-33]
 [-2.39116835e-33  1.00000000e+00]]


Por errores de punto flotante, la potencia a la 8 dio con residuos, lo corregimos haciendo una funcion que redondee elemento a elemento:


In [ ]:
def cleanMatrix(mat, decimals = 2):
  # Descripcion: funcion que "limpia" una matriz de errores de punto flotante

  # Inputs:
  # mat, objeto np.matrix
  # decimals: (int) cantidad de decimales a redondear, por defecto en 2

  # Outputs:
  # mat redondeada, objeto np matrix
  return np.matrix.round(mat, decimals)



In [ ]:
print(f"La matriz elevada a la 8 (limpia) es: \n {cleanMatrix(np.linalg.matrix_power(a, 8))}") # elevo la matriz a la 8

La matriz elevada a la 8 (limpia) es: 
 [[ 1.  0.]
 [-0.  1.]]


In [ ]:
# Parte 2

data = np.linalg.eig(a)
# buscar en documentacion la funcion "numpy.linalg.eig()" para entender que estructura retorna:

valores_propios = data[0]
vectores_propios = data[1]

print(f"los valores propios son: \n {valores_propios[0]} \n y: \n {valores_propios[1]}") # recordar que j es la unidad imaginaria
print(f"los vectores propios son:  \n {vectores_propios[0]}\n \n y \n \n {vectores_propios[1]} \n")




los valores propios son: 
 (0.7071067811865475+0.7071067811865475j) 
 y: 
 (0.7071067811865475-0.7071067811865475j)
los vectores propios son:  
 [[0.70710678+0.j 0.70710678-0.j]]
 
 y 
 
 [[0.-0.70710678j 0.+0.70710678j]] 



## Ejemplo: programacion lineal (ejercicio 7.6):
 Se dispone de un campo de 480 hectáreas, en el que se puede plantar maíz o trigo, en
superficies a definir. El maíz genera ganancias de $30/ha$, y el trigo $40/ha$. Por otra
parte, plantar una hectárea de maíz requiere 10 horas de tractor, una de trigo $20$ horas
de tractor, y el tractor está disponible un máximo de $8000$ horas. Hallar las superficies
óptimas a sembrar de cada grano para maximizar la ganancia.

## Solucion:

Consideramos:

* $x_1$ = "cantidad de hectáreas de maiz"
* $x_2$ = "cantidad de hectáreas de trigo"

El problema queda de la forma:


 $$ \text{max } 30x_1 + 40x_2$$

$$\text{Sujeto a:} \\  x_1+x_2 \leq 480 ; \\
10x_1+20x_2 \leq 8000; \\
x_1 \geq 0; \\
x_2 \geq 0;$$

Usamos la libreria CVXPY para resolver problemas de optimizacion

In [ ]:

import cvxpy as cp
import pandas as pd

print("Solvers instalados: ",cp.installed_solvers())


Solvers instalados:  ['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


Obligaremos a CVXPY a usar el solver 'GLPK' que usa SIMPLEX (ver documentacion)

Formulando el problema:

In [ ]:
x = cp.Variable(2, ) # Definimos vector (x, y) y ya lo seteamos en no negativo.


#-------------FORMULACION DE FUNCION OBJETIVO-------------#
c = np.array([30.0, 40.0]) # vector c
objective = cp.Maximize(c.T @ x) # para cvxpy el producto matricial es con @. (c.T es la traspuesta de c).

#---------------------------------------------------------#


#-------------FORMULACION DE CONSTRAINTS------------------#
A = np.matrix([
    [1  , 1]
,   [10 , 20]]) # matriz de desigualdades
b = np.array([
    480,
    8000]) # vector b
constraints = [A @ x <= b]
#---------------------------------------------------------#


#---------------MODELADO y SOLUCION-----------------------#

# Creacion del OBJETO problema
prob = cp.Problem(objective, constraints)




# Solucion del problema
result = prob.solve(solver = 'GLPK') # valor de la funcion objetivo, el vector solucion esta en "x.value". Le pasamos como parametro "GLPK" para que use ese solver.

print(f' Cantidad de hectareas de maiz:  {x.value[0]} \n Cantidad de hectareas de trigo: {x.value[1]} \n Presupuesto: ${result}')



 Cantidad de hectareas de maiz:  160.0 
 Cantidad de hectareas de trigo: 320.0 
 Presupuesto: $17600.0
